In [81]:
import pandas as pd
import re

In [94]:
# model

model = "m2"

In [95]:
x='test_data_w_'+model+'.csv'
x

'test_data_w_m2.csv'

In [96]:
df = pd.read_csv('test_data_w_'+model+'.csv')
df.head(2)

,Unnamed: 0.1,Unnamed: 0,Random ID,CRIS ID,Document ID,Domain,Date,Text,length,Code,...,date,TextContent,is_substring_fuzzy,StartIndex,EndIndex,document_length,ContextAroundSpan,start_index,end_index,SpanCat_Predictions
0,0,1,0.003589,10036220,45148545_Comments_Event,4,18/12/2015 00:00,stated journey was less difficult but still ch...,91,1,...,00:00.0,reported well in session stated journey was le...,True,7399.0,7445.0,612,reported well in session stated journey was le...,25,115,[]
1,1,5,0.004748,10246011,61089087_Comments_Event,4,04/02/2020 00:00,she said that she is able to manage her money ...,98,1,...,00:00.0,se lambeth prt cpa review dr zoë gray specialt...,True,0.0,31.0,12883,e the opportunity to discuss this in more deta...,200,297,[]


In [97]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Random ID', 'CRIS ID', 'Document ID',
       'Domain ', 'Date ', 'Text', 'length', 'Code', 'label', 'Self-harm',
       'Suicide', 'Physical', 'Historic', 'Dysfunctional social relationship',
       'Notes', 'Time taken to code (hours)', 'CN_Doc_ID', 'Brcid',
       'Cn_Doc_id', 'src_col', 'src_table', 'date', 'TextContent',
       'is_substring_fuzzy', 'StartIndex', 'EndIndex', 'document_length',
       'ContextAroundSpan', 'start_index', 'end_index', 'SpanCat_Predictions'],
      dtype='object')

In [98]:
df['SpanCat_Predictions_copy'] = df['SpanCat_Predictions']

In [99]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Random ID', 'CRIS ID', 'Document ID',
       'Domain ', 'Date ', 'Text', 'length', 'Code', 'label', 'Self-harm',
       'Suicide', 'Physical', 'Historic', 'Dysfunctional social relationship',
       'Notes', 'Time taken to code (hours)', 'CN_Doc_ID', 'Brcid',
       'Cn_Doc_id', 'src_col', 'src_table', 'date', 'TextContent',
       'is_substring_fuzzy', 'StartIndex', 'EndIndex', 'document_length',
       'ContextAroundSpan', 'start_index', 'end_index', 'SpanCat_Predictions',
       'SpanCat_Predictions_copy'],
      dtype='object')

In [100]:
# Split the data in 'span1' by commas into new columns
split_data = df['SpanCat_Predictions_copy'].str.split(',', expand=True)

In [101]:
column_names = []

for i in range(split_data.shape[1]):
    if i % 2 == 0:
        column_names.append(f'span{i//2 + 1}')
    else:
        column_names.append(f'label{i//2 + 1}')
 
split_data.columns = column_names

In [102]:
# Remove [ ( ) ] ' and spaces from the beginning and end of each split column

def clean_text(text):
    if pd.isna(text):  # If the value is NaN, return it as is
        return text
    return re.sub(r"[\[\]()']", '', text).strip()

for column in split_data.columns:
    split_data[column] = split_data[column].apply(clean_text)

In [103]:
# Concatenate the original dataframe with the split data

df = pd.concat([df, split_data], axis=1)

In [104]:
df.head(2)

,Unnamed: 0.1,Unnamed: 0,Random ID,CRIS ID,Document ID,Domain,Date,Text,length,Code,...,document_length,ContextAroundSpan,start_index,end_index,SpanCat_Predictions,SpanCat_Predictions_copy,span1,label1,span2,label2
0,0,1,0.003589,10036220,45148545_Comments_Event,4,18/12/2015 00:00,stated journey was less difficult but still ch...,91,1,...,612,reported well in session stated journey was le...,25,115,[],[],,None,None,None
1,1,5,0.004748,10246011,61089087_Comments_Event,4,04/02/2020 00:00,she said that she is able to manage her money ...,98,1,...,12883,e the opportunity to discuss this in more deta...,200,297,[],[],,None,None,None


In [105]:
df.to_csv('test_data_'+model+'_clean.csv', index=False)